In [31]:
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from copy import deepcopy
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split

In [32]:
random.seed(42)

In [33]:
def calc_accuracy(y_true, y_pred, metric='mae'):
    if metric == 'mae':
        acc = mean_absolute_error(y_true, y_pred)
    elif metric == 'r2':
        acc = r2_score(y_true, y_pred)
    elif metric == 'rmse':
        acc = root_mean_squared_error(y_true, y_pred)

    return acc

## Consensus methods

In [34]:
from consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor, HyperoptSearchRegressor

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge

In [35]:
method_list = [(SystematicSearchRegressor(cons_size=10, metric='r2'), 'Best'),
               #(SystematicSearchRegressor(cons_size=10**3, metric='r2'), 'All'),
               (RandomSearchRegressor(cons_size=2, n_iter=5, metric='r2'), 'Random'),
               (SystematicSearchRegressor(cons_size=2, metric='r2'), 'Systematic'),
               (GeneticSearchRegressor(cons_size=3, mut_prob=0.5, metric='r2'), 'Genetic'),
               #(HyperoptSearchRegressor(cons_size=3, n_iter=200, metric='rmse'), 'Hyperopt')
               ]

## Consensus comparison
### Input/output data path

In [36]:
bench_folder =  Path("benchmark_model_prediction/acf").resolve()

### Consensus building

In [37]:
estimators = [
    (LinearRegression(), 'LinearRegression'),
    (RandomForestRegressor(), 'RandomForestRegressor'),
    (MLPRegressor(), 'MLPRegressor'),
    (Ridge(), 'Ridge'),
    (SVR(), 'SVR'),
    (KNeighborsRegressor(), 'KNeighborsRegressor')
]

In [38]:
# res_df_val = pd.DataFrame()
res_df_test = pd.DataFrame()
res_df_train = pd.DataFrame()

In [39]:

for bench_name in tqdm(os.listdir(bench_folder)[:]):

    # load data
    # df_val = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_val.csv"))
    df_test = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_test.csv"))
    df_train = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_traincv.csv"))
    
    # remove y_true column prof predictions table
    # x_val, y_val = df_val.iloc[:, 1:], df_val.iloc[:, 0]
    x_train, y_train = df_train.iloc[:, 1:], df_train.iloc[:, 0]
    x_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)

    # build consensus
    for method_func, method_name in method_list:
        cons = method_func.run(x_train, y_train)
        y_pred = x_train[cons].mean(axis=1)
        #
        res_df_train.loc[bench_name, method_name] = calc_accuracy(y_train, x_train[cons].mean(axis=1), metric='r2')
        res_df_test.loc[bench_name, method_name] = calc_accuracy(y_test, x_test[cons].mean(axis=1), metric='r2')
        
        # # stacking model
        # x_stack, y_stack = df_train.iloc[:, 1:], df_train.iloc[:, 0]

    for model, name in estimators:
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        # predictions = cross_val_predict(model, x_train, y_train, cv=cv, n_jobs=-1) 
        res_df_test.loc[bench_name, f"Stacking_test_{model}"] = calc_accuracy(y_test, predictions, metric='r2')
    
    

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\micha\anaconda3\envs\mlstack\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 1/1 [00:08<00:00,  8.31s/it]


In [40]:
res_df_test

,Best,Random,Systematic,Genetic,Stacking_test_LinearRegression(),Stacking_test_RandomForestRegressor(),Stacking_test_MLPRegressor(),Stacking_test_Ridge(),Stacking_test_SVR(),Stacking_test_KNeighborsRegressor()
CHEMBL1862_Ki,0.786615,0.601987,0.783037,0.77183,0.799794,0.803841,0.784582,0.79983,0.791836,0.764608


## Meta statistics

In [41]:
from collections import Counter

### 1. Mean accuracy

In [42]:
res_df_val.mean(axis=0).round(2)

NameError: name 'res_df_val' is not defined

In [ ]:
res_df_test.mean(axis=0).round(2)

Best                                        0.64
Stacking_test_LinearRegression()        -6016.51
Stacking_test_RandomForestRegressor()       0.63
Stacking_test_MLPRegressor()            -3605.35
Stacking_test_Ridge()                   -5581.64
Stacking_test_SVR()                         0.50
Stacking_test_KNeighborsRegressor()         0.54
Random                                     -0.46
Systematic                                  0.62
dtype: float64

### 2. Top-N stat

In [ ]:
tmp = []
for dataset in res_df_test.index:
    min_i = res_df_test.loc[dataset].argmin()
    max_i = res_df_test.loc[dataset].argmax()
    #
    tmp.append(res_df_test.loc[dataset].index[max_i])
#
Counter(tmp)

Counter({'Stacking_test_RandomForestRegressor()': 13,
         'Best': 4,
         'Systematic': 1})

## 3. Pair comapison

In [ ]:
# tmp = res_df_test["Best"] - res_df_test["All"]
# tmp = res_df_test["Best"] - res_df_test["Systematic"]
# tmp = res_df_test["All"] - res_df_test["Systematic"]
# tmp = res_df_test["Systematic"] - res_df_test["Genetic"]
# tmp = res_df_test["Systematic"] - res_df_test["Stacking"]
tmp = res_df_test["Genetic"] - res_df_test["Stacking"]


res_pair = Counter(np.where(tmp > 0, 1, 2))
res_pair = {k.item():v for k, v in res_pair.items()}
print(dict(sorted(res_pair.items())))

KeyError: 'Genetic'

In [ ]:
team_1 = ["Res", "All", "Systematic"]
team_2 = ["Genetic", "Stacking"]

res = {1:0, 2:0}
for dataset in res_df_test.index:
    stats = res_df_test.loc[dataset].to_dict()
    alg = max(stats, key=stats.get)
    if alg in team_1:
        res[1] += 1
    else:
        res[2] += 1
res